# Interpretacja wyników i analiza danych

Do tej pory poznaliśmy narzędzia wykorzystywane w analizie danych. 
Tym razem skupimy się na analizie zbioru ocen i określimy, które czynniki mają wpływ na ocenę i średnią.

Zbiór danych do analizy: https://www.kaggle.com/datasets/rabieelkharoua/students-performance-dataset

In [1]:
# załadownaie pakietów
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# puść ten kod, 
# jeżeli wywołujesz plik  w folderze rozwiąznaia, 
# a ramka danych znajduje się w folderze data
import os 
os.chdir('../')

In [3]:
# Wczytanie zbioru
df = pd.read_csv("data/Student_performance_data.csv")

In [ ]:
# Nagłówek
df.head()

In [ ]:
# Przypomnijmy podstawowe statystyki
df.describe()

In [ ]:
# korelacja
plt.figure(figsize=(10,8))
sns.heatmap(round(df.corr(method= 'spearman'),2), annot =True, linewidths= 0.1)
plt.show()

Z analizy wynika, że mamy silną korelację pomiędzy liczbą nieobecności a średnią

## Badanie wpływu zmiennych na średnią ocen GPA

In [ ]:
# Średnia ocena dla osób z korepetycjami i bez
df.groupby('Tutoring').describe()['GPA']

In [ ]:
# Wykres gęstości
sns.kdeplot(df, x='GPA', hue = 'Tutoring', fill = True, common_norm= False)
plt.show()

In [ ]:
# Box plot
sns.boxplot(df, y= 'GPA', hue = 'Tutoring', fill = True)
plt.show()

In [ ]:
# Przeprowadzimy analizę wpływu zmiennych 'Tutoring','ParentalSupport','Extracurricular','ParentalEducation' na średnią
vars = ['Tutoring','ParentalSupport','Extracurricular','ParentalEducation']
for i in vars:
    print("#########################")
    print(f'Start analysis for {i}')
    print(df.groupby(i).describe()['GPA'])
    fig, ax = plt.subplots(ncols=2, nrows =1 )
    sns.kdeplot(ax = ax[0], data = df, x= 'GPA', hue = i, fill = True, common_norm= False)
    sns.boxplot(ax = ax[1], data= df, y = 'GPA', hue = i, fill = True)
    plt.title(f'GPA per {i}')
    plt.show()
    print(f'End analysis for {i}')
    print("#########################")

Z powyższych wykresów widzimy, że rozkłady i średnia GPA są zróżnicowane według zmiennej Tutor, Extracurricular oraz Parental Support. W przypadku zmiennej Parental Education zależność jest nieliniowa. Można zmienić enkodowanie zmiennej, co wprowadziłoby monotoniczonść, jednak utracilibyśmy sens typów edukacji.
Dla uczniów korzystających z korepetycji (tutoring=1) średnia ocen jest wyższa o około 16% (kwartyle również wskazują na wyższą średnią).
Dla uczniów korzystających z dodatkowych zajęć (Extracurricular=1) średnia ocen jest wyższa o około 10% (kwartyle również wskazują na wyższą średnią).


Wzrost parental support w badanej grupie powoduje wzrost średniej, można więc wnioskować, że warto wspierać dzieci w trakcie ich edukacji. Sprawdźmy średni wzrost.

In [ ]:
# Różnica wierszy
df.groupby('ParentalSupport').describe()['GPA']['mean'].diff(1)

In [ ]:
# Wykres różnicy w Średniej
df.groupby('ParentalSupport').describe()['GPA']['mean'].diff(1).plot()
plt.ylim(0,0.3)
plt.show()

Największą różnicę w wartościach bezwzględnych widać pomiędzy grupą 0 a 1, kolejne wykazują stały wzrost.

In [ ]:
# Wypisanie różnicy na ekranie
df_group = df.groupby('ParentalSupport').describe()['GPA']['mean']
df_group

In [ ]:
value = round(df_group[4] - df_group[0],2)
print(f'Różnica pomiędzy grupą bez wsparcia a z największym wynosi {value}.')

Wiemy już jaki wpływ ma wsparcie rodziców na wyniki w jednostkach bezwzględnych. Możemy teraz sprawdzić jak się ona kształtuje procentowo.

In [ ]:
changes = df.groupby('ParentalSupport').describe()['GPA'][['mean']]
changes

In [ ]:
changes['previous_mean'] = changes['mean'].shift(1)
changes

In [ ]:
changes['percentage_change'] = changes['mean'] / changes['previous_mean'] - 1
changes

In [ ]:
changes['percentage_change'].plot()
plt.ylim(0,0.15)
plt.show()

Powyższy wykres potwierdza, że największą różnicę w przeciętnej wartości średniej widać pomiędzy grupą bez wsparcia a ze wspraciem.

## Analiza ocen
### Zmienne nominalne

In [ ]:
# Rozklad wieku
sns.kdeplot(df, x = 'Age', hue = 'GradeClass', fill= True, common_norm= False)
plt.show()

## Zmienne kategoryczne

In [20]:
# Dodanie stalej do ramki danych
df['denominator'] = 1

In [ ]:
# Pivot table - tabela przestawna
pivot_summary = pd.pivot_table(data = df, index = 'ParentalSupport', columns = 'GradeClass', aggfunc='sum', margins=True)['denominator']
pivot_summary

In [ ]:
pivot_summary.loc['All',:]

In [ ]:
# Róznice procentowe
df_percentage = round(pivot_summary / pivot_summary.loc['All',:],2).reset_index()
df_percentage

In [ ]:
# Wizualizacja wyników procentowych
for i in range(5):
    plt.bar(x=[0,1,2,3,4], height = df_percentage.loc[0:4,float(i)])
    plt.title(f'Rozklad wsparcia rodzicielskiego dla oceny {i}')
    plt.show()

Powyższe wykresy wykazują róznicę pomiędzy grupą bez wsparcia a ze wsparciem. Pomiędzy poszczególnymi grupami wsparcia rozkład cen jest podobny.